# Iframe Widget POC

Testing embedding a simple HTML widget in Jupyter via iframe.

## Setup

First, start a simple HTTP server in another terminal:
```bash
cd examples/jupyter-iframe-poc
python3 -m http.server 8888
```


In [5]:
# Method 1: Direct IFrame
from IPython.display import IFrame

IFrame('http://localhost:9999/simple_widget.html', width=600, height=300)


In [6]:
# Method 2: Toggle Button (show/hide with click)
from IPython.display import display, HTML
import uuid

def symbol_palette(url='http://localhost:9999/simple_widget.html', width=600, height=300):
    """Create a symbol palette that can be shown/hidden."""
    wid = f"w-{uuid.uuid4().hex[:8]}"
    
    return HTML(f"""
    <button onclick="
        var w = document.getElementById('{wid}');
        w.style.display = w.style.display === 'none' ? 'block' : 'none';
        this.textContent = w.style.display === 'none' ? '📐 Open Symbol Palette' : '❌ Close';
    " style="padding: 10px 20px; font-size: 14px; cursor: pointer;">
        📐 Open Symbol Palette
    </button>
    <div id="{wid}" style="display: none; margin-top: 10px;">
        <iframe src="{url}" width="{width}" height="{height}" 
                style="border: 1px solid #ccc; border-radius: 4px;"></iframe>
    </div>
    """)

display(symbol_palette())


In [7]:
# Method 3: With message passing (iframe sends result back to notebook)
from IPython.display import display, HTML
import uuid

def interactive_palette(url='http://localhost:9999/simple_widget.html', width=600, height=350):
    """Palette that sends results back to Python."""
    wid = f"w-{uuid.uuid4().hex[:8]}"
    rid = f"r-{uuid.uuid4().hex[:8]}"
    
    return HTML(f"""
    <button onclick="
        var w = document.getElementById('{wid}');
        w.style.display = w.style.display === 'none' ? 'block' : 'none';
        this.textContent = w.style.display === 'none' ? '📐 Open' : '❌ Close';
    " style="padding: 10px 20px; cursor: pointer;">📐 Open</button>
    
    <div id="{wid}" style="display: none; margin-top: 10px;">
        <iframe src="{url}" width="{width}" height="{height}" 
                style="border: 1px solid #ccc;"></iframe>
    </div>
    
    <div id="{rid}" style="margin-top: 10px; padding: 10px; background: #e8f5e9; display: none;">
        <strong>Received:</strong> <span id="{rid}-text"></span>
    </div>
    
    <script>
        window.addEventListener('message', function(e) {{
            if (e.data && e.data.type === 'widget-result') {{
                document.getElementById('{rid}').style.display = 'block';
                document.getElementById('{rid}-text').textContent = e.data.content;
                
                // Send to Python kernel
                if (typeof Jupyter !== 'undefined' && Jupyter.notebook) {{
                    Jupyter.notebook.kernel.execute("_result = '" + e.data.content + "'");
                }}
            }}
        }});
    </script>
    """)

display(interactive_palette())


In [8]:
# Check if we received a result from the widget
try:
    print(f"Result from widget: {_result}")
except NameError:
    print("No result yet - click 'Send to Jupyter' in the widget above")


No result yet - click 'Send to Jupyter' in the widget above
